# VGG16 Transfer Learning for MNIST Dataset

## Import libraries

In [3]:
from keras.applications.vgg16 import VGG16
import tensorflow as tf 
import numpy as np 
import pandas as pd 

## Load VGG16 pretrained model 
Param: 
* `include_top`: False, since we're experiment how the network is working but input tensor `(3,224,224)` so we've to decided to remove all Fully-Connected layers
* `weights`: use `imagenet` 
* `input_tensor`: (28,28,1)
* `classes`: 10 classes
* `classifier_activation` : `softmax` since classification problem

In [ ]:
# attach FC1 (28,28,3) -> eval 
# attach FC1 -> FC2 -> eval

## Load Dataset


In [4]:
data_train = pd.read_csv(r"./data/train_new.csv") 
data_test = pd.read_csv(r"./data/test_new.csv") 
data_train.head()
data_test.head()


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training

##  Generate submission